# TP2 - Spatial Filtering
ATRIM - Option Datasim

Ecole Centrale Nantes

Diana Mateus

Edouard GAUTRON et Maria clara VALLE

## Introduction

Dans ce rapport, nous explorerons les principes du filtrage spatial linéaire appliqué aux images 2D. Nous utiliserons différents types de filtres (kernel) et examinerons leurs capacités à accomplir diverses opérations sur des images en ajustant leurs paramètres. Cette première partie permettra de mieux comprendre comment ces filtres modifient le contenu visuel et leur utilité dans des tâches comme le lissage, le renforcement des détails, ou encore la détection de contours.

Dans une deuxième phase, nous implémenterons des algorithmes de filtrage plus avancés afin de développer des méthodes capables de reconnaître des motifs spécifiques dans les images tout en préservant les contours et les détails essentiels.

In [ ]:
import skimage.io as io
import matplotlib.pyplot as plt
import numpy as np
import os
import math
from skimage.restoration import denoise_bilateral
from skimage.transform import resize, rescale
from scipy import ndimage
from scipy import signal


In [ ]:
IMDIR = r"C:\Users\maria\DataSim\ATRIM\TP2-ATRIM-Spatial Filtering\ATRIM-TP2\images" #nous indiquon le chemin pour acceder aux images


# 1 Linear spatial filtering with convolution




Nous commençons par exécuter l'exemple "meanKernel" qui crée un kernel flou.


In [ ]:
def meanKernel(hs):
    #hs on defini la taille du kernel
    #hs est un entier de la moitier de la taille du kernel
    #on créer un filtre carré de taille 2*hs+1
    kernel = np.zeros((hs*2+1,hs*2+1))
    kernel += 1/(hs*2+1)**2
    return kernel

#on affiche les propriétés
width=12
height=3
plt.rcParams['figure.figsize'] = [width, height]

#on crée et affiche trois differents kernels
k = 1
for hs in [1,3,11]:
    plt.subplot(1,3,k)
    kernel = meanKernel(hs)
    plt.imshow(kernel, vmin=0, vmax=0.2)
    plt.title(f'Mean, hs ={hs}')
    plt.colorbar()
    k+=1
plt.show()


Ensuite, nous le convoluons avec une image en utilisant la fonction convolve. Avec ça, on va observer l'effet que chaque filtre moyenne a sur les images



In [ ]:
f = os.path.join(IMDIR, "grass.jpg")

#Display properties
width=10
height=5

#Filter parameters
hs = 3 #change le niveau de blur
sigma = 2

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = resize(im,(100,100))

# Define filter and convolve
kernel = meanKernel(hs)
im_filtered_scipy = ndimage.convolve(im,kernel)


# on affiche l'image originale
fig=plt.figure(figsize=(width, height))
plt.subplot(1,2,1)
plt.imshow(im, cmap = 'gray')
plt.title('Original')

# affichage de l'image resultante
plt.subplot(1,2,2)
plt.imshow(im_filtered_scipy, cmap = 'gray')
plt.title('Mean scipy conv')


In [ ]:
f = os.path.join(IMDIR, "grass.jpg")

#Display properties
width=10
height=5

#Filter parameters
hs = 3 #change le niveau de blur
sigma = 2

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = resize(im,(100,100))

# Define filter and convolve
kernel = meanKernel(hs)
im_filtered_scipy = ndimage.convolve(im,kernel)

# on affiche l'image originale
fig=plt.figure(figsize=(width, height))
plt.subplot(1,2,1)
plt.imshow(im, cmap = 'gray')
plt.title('Original')

# affichage de l'image resultante
plt.subplot(1,2,2)
plt.imshow(im_filtered_scipy, cmap = 'gray')
plt.title('Mean scipy conv')


On voit que quand on augmente la taille du filtre (hs), l'image devient plus flou

## 1.2. Gaussian Kernel
Ici, nous créons des noyaux gaussiens de tailles et d'écart-types variés, puis nous les appliquons à l'image "grass.png"





In [ ]:


def gaussianKernel(hs,sigma, normalize=True): 
    # pour crée le kernel on crée une matrice de zero de la meme taille
    kernel = np.zeros((hs*2+1,hs*2+1))
    ax = np.arange(-hs, hs+1)

    # on crée le kernel
    xx, yy = np.meshgrid(ax, ax)

    #on definit le kernel en s'appuyant sur le model du cours;
    kernel = np.exp(-(xx**2+yy**2)/(2*sigma**2))/(2*np.pi*sigma**2) 
    

    # on normalise et retourne
    if normalize:
        kernel = kernel / np.sum(kernel)

    return kernel

In [ ]:
f = os.path.join(IMDIR, "grass.jpg")

#Display size
width=10
height=10
k = 1
iter_conv = 1
filters = []

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = resize(im,(100,100))

# Display the original image
fig=plt.figure(figsize=(3, 3))
plt.imshow(im, cmap = 'gray')
plt.title('Original')

# Don definit les differents parametres pour les differents filtres
filter_sizes=[1,3,5]
sigma_values=[0.1,1,2]

# dans un premier temps on affiche les filtres
fig=plt.figure(figsize=(width, height))

####### debut de notre code
for hs in filter_sizes:
    for sigma in sigma_values:
        filter = gaussianKernel(hs,sigma, normalize=True)
        plt.subplot(3,6,2*k-1)
        #display the filter
        plt.imshow(filter, cmap = 'gray')
        plt.title(f"filtre, hs: {hs}, sigma : {sigma}")
        print(f'sum des element kernel avec hs = {hs} et sigma = {sigma} : {np.sum(filter)}')

        plt.subplot(3,6,2*k)
        # Convolve and display the filtered image
        conv = ndimage.convolve(im,filter)#on fait la convolution pour appliquer les filtres aux images
        plt.imshow(conv, cmap = 'gray')
        plt.title("image filtrée")
        
        k += 1
        filters.append(filter)

plt.show()

# Convolve the image with the filter and display the filtered image
#fig=plt.figure(figsize=(width, height))

#for kernel in filters:
    #print(kernel)
    #print('prochaine')
    #print(np.sum(kernel[::2]))
   




 Ainsi, on peut observer l'affichage des trois filtres, qui n'ont pas tous la meme taille ni le meme sigma, 
 on s'attend donc a des resultats differents.
 On observe que plus la taille du filtre est grande et le sigma grand plus l'image est floue

<font color='red' size = '5'> Answer to question </font>On voit bien que la somme des éléments du noyau est égale à 1, ce qui est nécessaire car cela permet de conserver la luminosité de l'image.

## 1.3 Filtering with your own Convolution

Nous avons répété le lissage effectué précédemment en implémentant notre propre fonction de convolution. Cette fonction prend en entrée une image et un noyau de filtre (matrice de poids) pour renvoyer l'image filtrée. Après avoir appliqué cette méthode sur les mêmes images que celles utilisées avec la fonction intégrée de Scikit-learn, nous avons comparé les résultats. 

<font color='red' size = '5'> Answer to question c) </font>** **Write down your findings**, notably the reasons for any possible difference with the in-built implementation.

Les images obtenues avec notre implémentation étaient très similaires à celles produites par Scikit-learn, confirmant ainsi la validité de notre approche. Toutefois, on a regarde la difference entre l'image filtré a partir de notre convolution et cela déjà implementé. Pour cela on applique les deux methode et on soustrait les deux matrices obtenues, pour mettre en valeure des differences
Apres avoir fait cette difference on remarque une image noir ce qui montre qu'il n'y a pas de difference, a part sur les bords 
cela provient probablement de impadded

<font color='red' size = '5'> Answer to question d) </font> Why and how can the convolution can be written as a matrix multiplication? why is it interesting?

on remarque que la convolution effectue des opperation tres proche de celle de a multiplication de matrice juste on ne multipli et addition pas les elements dans le meme orde il suffit alors de modifier les matrices pour les quelles on faisait la convolution et ainsi une simple multiplication de matrice suffira. Ce procedé est moins couteu ce qui le rend plus interessant


In [ ]:
def myConvolution(imsource,kernel):

    # Find image and kernel sizes
    im_shape = imsource.shape
    imh,imw = im_shape[0], im_shape[1]
    kh,kw = kernel.shape
    print(kh)
    delta_h=int((kh-1)/2)
    delta_w=int((kw-1)/2)

    print(delta_h)
    # on elargie l'image avec des zeros autour car le filtre a une certaine taille qui depasse donc de la matrice si il passe le long des bords
    imPadded = np.zeros((imh+2*delta_h,imw+2*delta_w))
    imPadded[delta_h:imh+delta_h,delta_w:imw+delta_w] = imsource
    

    # Create an empty image to store the result
    imDest = np.zeros((imh,imw))

    
    for i in range(imh):
        for j in range(imw): # on parcour la matrice par les lignes et colones

####### On a essayé une premiere methode qui fonctionne mais pas parfaitement on a choisit d'en faire une deuxieme 
             #for u in range(kh):
             #   for v in range(kw):

                   # imDest[i,j] += kernel[u , v]*imPadded[i +2*delta_h- u, j +2*delta_h- v]

            
###### deuxieme methode
            imPatch = imPadded[i:i+kh, j:j+kw]
            imDest[i,j] = np.sum(imPatch*np.flip(kernel))


    #END FILL IN
    return imDest


In [ ]:
f = os.path.join(IMDIR, "grass.jpg")

#Display properties
width=10
height=10

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = resize(im,(100,100))

# Display the original image
fig=plt.figure(figsize=(width, height))
plt.subplot(1,4,1)
plt.imshow(im, cmap = 'gray')
plt.title('Original')

# Define filter parameters
hs = 11
sigma = 2
kernel = gaussianKernel(hs,sigma)

##### debut de notre code

# Convolve and display the filtered image
conv = myConvolution(im,kernel)
fig=plt.figure(figsize=(width, height))
plt.subplot(1,4,2)
plt.imshow(conv, cmap = 'gray')
plt.title('My_conv')

conv_2 = ndimage.convolve(im,kernel)
plt.subplot(1,4,3)
plt.imshow(conv, cmap = 'gray')
plt.title('Convole')

dif = conv_2 - conv

plt.subplot(1,4,4)
plt.imshow(dif, cmap = 'gray')
plt.title('Convole')

#### fin de notre code


 Maintenant on va comparer notre solution pour appliquer le filtre avec la convolution
 pour cela on applique les deux methode et on soustrait les deux matrices obtenues, pour mettre en valeure des differences
 Apres avoir fait cette difference on remarque une image noir ce qui montre qu'il n'y a pas de difference, a part sur les bords 
 cela provient probablement de impadded

## On etudie les filtres derivés


On a défini la fonction de noyau nécessaire et l'a convoluée avec les images AscentB ou Moon dans le dossier enhance pour obtenir :

l'image du gradient dans la direction horizontale,
l'image du gradient dans la direction verticale,
le Laplacien de l'image,
l'image améliorée après l'ajout des "détails" du Laplacien (et la normalisation).

In [ ]:
# Pour se faire on créer differents kernels associé a chaque opération

def sobel_x():
    kernel = np.zeros((3,3))
# on crée le filtre    
    kernel = [[1, 0, -1],
             [2, 0, -2],
             [1 ,0, -1]]
    return kernel

def sobel_y():
    kernel = np.zeros((3,3))
# on crée le filtre
    kernel = [[1, 2, 1],
             [0, 0, 0],
             [-1 ,-2, -1]]
    return kernel


def laplacian():
# on crée le filtre
    kernel = [[0, 1, 0],
             [1, -4, 1],
             [0, 1, 0]]

    return kernel

def normalize(im):
    im = (im-im.min())/(im.max()-im.min())
    return im


In [ ]:
f = os.path.join(IMDIR, "enhance/moon-blurred.tif")
#f = os.path.join(IMDIR, "ascentB.png")

#Display properties
width=15
height=5

# Read and preprocess image
im = io.imread(f, as_gray=True)
im = im.astype(float)
im = normalize(im)
#im = resize(im,(100,100))

# Display the original image
fig=plt.figure(figsize=(width, height))
plt.subplot(1,5,1)
plt.imshow(im, cmap = 'gray')
plt.title('Original')

##### debut code alouté 
# Convolve and display the filtered and the enhanced image
im_filtered_x =  ndimage.convolve(im,sobel_x())
im_filtered_x =  np.clip(im_filtered_x, 0, 1) #sature l'image entre 0 et 1
im_filtered_y =  ndimage.convolve(im,sobel_y())
im_filtered_y =  np.clip(im_filtered_y, 0, 1)
im_filtered_laplace =  ndimage.convolve(im, laplacian())
im_filtered_laplace  =  np.clip(im_filtered_laplace, 0, 1)

plt.subplot(1,5,2)
plt.imshow(im_filtered_x, cmap = 'gray')
plt.title('gradx')
plt.subplot(1,5,3)
plt.imshow(im_filtered_y, cmap = 'gray')
plt.title('grady')

norme = np.sqrt(im_filtered_x**2+im_filtered_y**2)
plt.subplot(1,5,4)
plt.imshow(norme, cmap = 'gray')
plt.title('magnitude')

plt.subplot(1,5,5)
plt.imshow(im_filtered_laplace, cmap = 'gray')
plt.title('laplace')
##### fin



 On observe differents resultatts selon les filtres utilisés notaments selon le sens de la derivé qui ne mets pas en valeur les memes particumarités 
 de la lune l'un les bords et l'autre les crateres, on remarque que pour le cas de la lune le laplacien efface les crateres et nelaisse place qu'au contour


# 2 Non linear filtering


## 2.1 Correlation: Finding Charlie

On a utilisé la corrélation croisée normalisée (NCC) par blocs pour trouver automatiquement Waldo (Charlie) dans une image. Pour cela, on a recherché l’image modèle (charlie-template) dans les images marche-crop. Il a également été utile de créer un notebook séparé (Finding Charlie) pour cette tâche, car elle consomme beaucoup de mémoire.

On a évalué l'expression du NCC issue des diapositives (décrite dans le filtrage avancé non-local means) pour comparer l'image modèle à chaque position dans l'image cible. Les résultats ont été stockés, et on a récupéré l'emplacement ayant obtenu le score NCC le plus élevé. Enfin, on a tracé cet emplacement sur l'image cible.

<font color='red' size = '5'> Answer to question </font>On vérifie que la méthode implementé a quelques nécessite de quelques hipotheses, par exemple le fact qu'on suppose que l’image modèle (charlie) est présente dans l'image cible et qu'elle est suffisamment similaire en termes de taille, orientation et luminosité pour que la corrélation croisée puisse détecter de correspondance.Aussi, on fait l'hypothèse que les niveaux de lumière dans l'image cible et dans l'image modèle sont homogènes. 

Comme limitation, on a que le charlie template doit être dans la même position dans l'image du marche, c.a.d que la méthode ne supporte pas de rotation. Aussi, l'algoritime est trés lent.



## 2.2 Bilateral Filter  

On a implémenté notre propre version du filtre bilatéral et on a comparé ses résultats avec ceux de la fonction denoise_bilateral de Scikit-learn.

 <font color='red' size = '5'> Answer to question a) </font> compare its results vs. scikit ``denoise_bilateral`` function.

L'implémentation a permis d'observer les différences de performance et de qualité entre les deux méthodes. Les résultats de notre filtre bilatéral ont montré une capacité à préserver les contours tout en réduisant le bruit. Les paramêtres des deux fonctions sont différents, et donc c'est difficile de regarder des vrais differénces.

<font color='red' size = '5'> Answer to question b) </font> Compare and comment the bilateral results versus the mean and gaussian filter for the ``zebra`` group of images of the ``bilateral`` folder

Ensuite, on a comparé les résultats du filtre bilatéral avec ceux du filtre moyen et du filtre gaussien sur le groupe d'images de zèbres dans le dossier bilatéral. Les observations ont révélé que le filtre bilatéral était plus efficace pour conserver les détails des contours, tout en removent le bruit de différents types, contrairement au filtre moyen, qui tend à lisser l'image de manière plus uniforme, entraînant une perte de détails. Le filtre gaussien, quant à lui, offrait également un bon lissage, mais avait tendance à créer un flou principalement dans les contours. En résumé, le filtre bilatéral s'est avéré plus adapté pour les images contenant des détails fins tout en maintenant un bon équilibre entre réduction de bruit et préservation des contours.


In [ ]:
#poprieté pour comparation
def MSE(noisy_image, ref_image):
    return np.mean((ref_image - noisy_image)**2)

def PSNR(noisy_image, ref_image):
    max = np.max(noisy_image)
    MSE_ = MSE(noisy_image, ref_image)
    PSNR = 20*np.log(max/np.sqrt(MSE_))
    return PSNR

In [ ]:
from skimage.restoration import denoise_bilateral

#ouvrir les archives de zebra
f_bruit = os.path.join(IMDIR, "bilateral\zebra\zebra_speckle_2.png") #changer le bruit voulu
f_ref = os.path.join(IMDIR, "bilateral\zebra/ref.jpg")

width=15
height=5

#lire l'image, convertir et normaliser
im = io.imread(f_bruit, as_gray=True)
im = im.astype(float)
im = normalize(im)

im_ref = io.imread(f_ref, as_gray=True) 
im_ref = im_ref.astype(float)
im_ref = normalize(im_ref)

#parametres du filtre
hs = 3
sigma1 = 1
sigma2 = 2

#notre version du filtre bilateral
def bilateral(im, hs, sigma1, sigma2):
    bilateral_filter = np.zeros((2*hs+1, 2*hs+1))
    grid = np.arange(-hs, hs+1)
    u, v = np.meshgrid(grid, grid)
    bilateral_filter = np.exp(-(u**2+v**2)/(2*sigma1**2))*np.exp(-(im[u,v]-im[0,0])**2/(2*sigma2**2))
    
    #normalizer
    bilateral_filter = bilateral_filter/np.sum(bilateral_filter)
    
    return bilateral_filter

#création du filtre
bilateral_myversion = bilateral(im, hs, sigma1, sigma2) 

#application du filtre
im_filtered_myversion = ndimage.convolve(im, bilateral_myversion)
#test pour la deuxiéme itération
im_filtered_myversion_2 = ndimage.convolve(im_filtered_myversion, bilateral_myversion)

#comparation avec filtre scikit-------------------------------------------------------------------------------
#application du filtre scikit
im_filtered_scipy = denoise_bilateral(im, win_size=hs*2+1, sigma_color=0.3, sigma_spatial=20)
#test pour la deuxiéme itération
im_filterd_scipy_2 = denoise_bilateral(im_filtered_scipy, win_size=hs*2+1, sigma_color=0.05, sigma_spatial=15)

#comparation avec mean et gaussian filtre-------------------------------------------------------------------------
#mean filter
kernel = meanKernel(hs)
im_filtered_mean = ndimage.convolve(im,kernel)

#gaussian filter
gaussian = gaussianKernel(hs,sigma, normalize=True)
im_filtered_gaussian = ndimage.convolve(im,gaussian)


fig = plt.figure(figsize = (width, height))
plt.subplot(1,3,1)
plt.imshow(im_ref, cmap = 'gray')
plt.title('ref')

plt.subplot(1,3,2)
plt.imshow(im, cmap = 'gray')
plt.title('bruit')

plt.subplot(1,3,3)
plt.imshow(im_filtered_myversion, cmap = 'gray')
plt.title('My_bilateral')
plt.show()

#test 2 itérations de filtrage -------------------------------------------------------
fig = plt.figure(figsize = (width, height))
plt.subplot(1,4,1)
plt.imshow(im_filtered_myversion_2, cmap = 'gray')
plt.title('2 itération My_bilateral')


#comparation scikit ----------------------------------------------------------------
plt.subplot(1,4,2)
plt.imshow(im_filtered_scipy, cmap = 'gray')
plt.title('Bilateral scikit')

#comparation mean ----------------------------------------------------------------
plt.subplot(1,4,3)
plt.imshow(im_filtered_mean, cmap = 'gray')
plt.title('Mean filtre')


#comparation gaussian ----------------------------------------------------------------
plt.subplot(1,4,4)
plt.imshow(im_filtered_gaussian, cmap = 'gray')
plt.title('Gaussian filtre')

plt.show()

#comparation des filtrages pour le valeur de PSNR 
print(f'PSNR My_bilateral:{PSNR(im_filtered_myversion, im_ref)}\nPSNR Scipy: {PSNR(im_filtered_scipy, im_ref)} ')


## Conclusion
Nous avons exploré plusieurs approches, notamment le filtre bilatéral, le filtre moyen, le filtre gaussien et les filtres dérivés, ainsi que la corrélation croisée normalisée (NCC) pour la détection d'objets. 

Nous avons tout d'abord analysé le filtre moyen et le filtre gaussien en appliquant différentes tailles de noyau et valeurs d'écart-type. Les résultats ont révélé que le filtre moyen est efficace pour lisser les images et réduire le bruit, mais il peut également provoquer une perte significative de détails, en particulier dans les zones à contraste élevé. En revanche, le filtre gaussien, avec ses différentes configurations de taille et d'écart-type, a montré une capacité à maintenir une meilleure définition des contours tout en atténuant le bruit. Cependant, un écart-type trop élevé ou une taille de noyau trop grande peuvent entraîner un flou excessif et une perte d'informations critiques.

Nous avons ensuite étudié les filtres dérivés, qui sont utilisés pour détecter les gradients dans les images. Ces filtres ont permis d'extraire les contours et les détails des images en mettant en évidence les variations d'intensité. 

De plus, nous avons implémenté notre propre version du filtre bilatéral, que nous avons comparée à la fonction intégrée denoise_bilateral de Scikit-learn. Nos résultats ont montré que le filtre bilatéral est particulièrement efficace pour réduire le bruit tout en préservant les contours et les détails, contrairement aux filtres moyen et gaussien.

Enfin, l'utilisation de la corrélation croisée normalisée (NCC) pour la détection automatique de Waldo (Charlie) a permis de mettre en évidence les hypothèses et les limites de cette méthode. Bien qu'elle soit efficace dans des conditions idéales, des changements de taille et de rotation dans l'image a chercher peuvent compromettre les résultats.

Globalement, ces expériences et analyses nous ont permis d'approfondir notre compréhension des techniques de filtrage et de détection d'objets en traitement d'images, tout en soulignant l'importance de choisir les méthodes et les paramètres appropriés pour obtenir des résultats optimaux.